In [1]:
import numpy as np

In [2]:
a=np.array([[1,0],[2,1]])
a=a.repeat(2,axis=1).repeat(2,axis=0)
b=np.array([[1,2],[3,4]])
b=np.tile(b,(2,2))
np.multiply(b,a)

array([[1, 2, 0, 0],
       [3, 4, 0, 0],
       [2, 4, 1, 2],
       [6, 8, 3, 4]])

In [6]:
a

array([[1, 1, 0, 0],
       [1, 1, 0, 0],
       [2, 2, 1, 1],
       [2, 2, 1, 1]])

In [9]:
a.reshape(16,1).shape

(16, 1)

In [21]:
p=4
m=3
su_=np.zeros((p,m))
for i in range(p):
    for j in range(m):
        if (i>=j):
            su_[i,j]=i+1-j 
su_=su_.repeat(2,axis=1).repeat(3,axis=0)
su_

array([[1., 1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0.],
       [2., 2., 1., 1., 0., 0.],
       [2., 2., 1., 1., 0., 0.],
       [2., 2., 1., 1., 0., 0.],
       [3., 3., 2., 2., 1., 1.],
       [3., 3., 2., 2., 1., 1.],
       [3., 3., 2., 2., 1., 1.],
       [4., 4., 3., 3., 2., 2.],
       [4., 4., 3., 3., 2., 2.],
       [4., 4., 3., 3., 2., 2.]])

In [54]:
np.tile(np.array([3,4]),(1,p))

array([[3, 4, 3, 4, 3, 4, 3, 4]])

In [1]:
import casadi

In [10]:
x = casadi.SX.sym('x'); y = casadi.SX.sym('y')
qp = {'x':casadi.vertcat(x,y), 'f':casadi.sin(x)**2+casadi.sin(y)**2, 'g':x+y-10}
S = casadi.qpsol('S', 'ipopt', qp)
print(S)
r = S(lbg=0)
x_opt = r['x']
print('x_opt: ', x_opt)

CasADi -  WARNING(".../casadi/core/plugin_interface.hpp:322: Assertion "handle!=nullptr" failed:
PluginInterface::load_plugin: Cannot load shared library 'libcasadi_conic_ipopt.dll': 
   (
    Searched directories: 1. casadipath from GlobalOptions
                          2. CASADIPATH env var
                          3. PATH env var (Windows)
                          4. LD_LIBRARY_PATH env var (Linux)
                          5. DYLD_LIBRARY_PATH env var (osx)
    A library may be 'not found' even if the file exists:
          * library is not compatible (different compiler/bitness)
          * the dependencies are not found
   )
  Tried 'e:\ProgramFiles\miniconda3\lib\site-packages\casadi' :
    Error code (WIN32): 126
  Tried '' :
    Error code (WIN32): 126
  Tried '.' :
    Error code (WIN32): 126") [.../casadi/core/plugin_interface.hpp:171]


RuntimeError: .../casadi/core/plugin_interface.hpp:417: Plugin 'ipopt' is not found.

In [52]:
H = casadi.DM(np.eye(2))
A = casadi.DM(np.ones((1,2)))
g = casadi.DM(np.zeros(2))
lba = 10.

qp = {}
qp['h'] = H.sparsity()
qp['a'] = A.sparsity()
S = casadi.conic('S','qpoases',qp)
print(S)





r = S(h=H, g=g, a=A, lba=lba)
x_opt = r['x']
print('x_opt: ', x_opt)

S:(h[2x2],g[2],a[1x2],lba,uba,lbx[2],ubx[2],x0[2],lam_x0[2],lam_a0,q[],p[])->(x[2],cost,lam_a,lam_x[2]) QpoasesInterface


####################   qpOASES  --  QP NO.   1   #####################

    Iter   |    StepLength    |       Info       |   nFX   |   nAC    
 ----------+------------------+------------------+---------+--------- 
       0   |   1.866661e-007   |   ADD CON    0   |     1   |     1   
       1   |   1.333333e-008   |   REM BND    1   |     0   |     1   
       2   |   1.000000e+000   |    QP SOLVED     |     0   |     1   
x_opt:  [5, 5]


In [22]:
from pyomo.environ import * 
# path = '~/Documents/ipopt/ipopt' # 这里的 path 指的就是刚刚 ipopt 执行文件的路径

model = ConcreteModel() # 创建模型对象

# define model variables
# domain = Reals(Default) / NonNegativeReals Binary
model.x1 = Var(domain=Reals)
model.x2 = Var(domain=Reals)
# define objective function
# sense = minimize(Default) / maximize 
model.f = Objective(expr = model.x1**2 + model.x2**2, sense=minimize)
# define constraints, equations or inequations 
model.c1 = Constraint(expr = -model.x1**2 + model.x2 <= 0)
model.ceq1 = Constraint(expr = model.x1 + model.x2**2 == 2)
# use 'pprint' to print the model information 
model.pprint()


2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    f : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x1**2 + x2**2

2 Constraint Declarations
    c1 : Size=1, Index=None, Active=True
        Key  : Lower : Body         : Upper : Active
        None :  -Inf : - x1**2 + x2 :   0.0 :   True
    ceq1 : Size=1, Index=None, Active=True
        Key  : Lower : Body       : Upper : Active
        None :   2.0 : x1 + x2**2 :   2.0 :   True

5 Declarations: x1 x2 f c1 ceq1


In [23]:
SolverFactory('ipopt', executable=r'E:\ProgramFiles\Ipopt-3.11.1\bin\ipopt.exe').solve(model).write()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.04690885543823242
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
